In [30]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
%matplotlib inline
import os
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
import seaborn as sns

In [5]:
#Read In First Equity Dataframe Prices
first_equity_rtns = pd.read_csv('Stock ETF data pull.csv')
first_equity_rtns.head()

,Date,VOOV,EZU,IWM,SMH,IBB,XLU,VOOG,SPY,QQQ
0,2015-02-11,89.86,37.22,119.45,55.28,105.75,46.08,101.52,206.93,104.78
1,2015-02-12,90.75,38.20,120.81,56.06,106.81,45.87,102.60,208.92,106.00
2,2015-02-13,91.08,38.29,121.53,56.39,107.82,45.17,103.12,209.78,106.91
3,2015-02-17,91.13,38.34,121.91,56.63,108.94,45.13,103.24,210.11,107.01
4,2015-02-18,91.01,38.44,122.04,56.44,109.81,46.20,103.44,210.13,107.16


In [6]:
#Dropping the First Row in the Equity DataFrame
equity_clean = first_equity_rtns.drop([0])
equity_clean

,Date,VOOV,EZU,IWM,SMH,IBB,XLU,VOOG,SPY,QQQ
1,2015-02-12,90.75,38.20,120.81,56.06,106.81,45.87,102.60,208.92,106.00
2,2015-02-13,91.08,38.29,121.53,56.39,107.82,45.17,103.12,209.78,106.91
3,2015-02-17,91.13,38.34,121.91,56.63,108.94,45.13,103.24,210.11,107.01
4,2015-02-18,91.01,38.44,122.04,56.44,109.81,46.20,103.44,210.13,107.16
5,2015-02-19,90.75,38.40,122.05,56.47,110.66,45.68,103.64,209.98,107.69
...,...,...,...,...,...,...,...,...,...,...
1252,2020-02-03,122.67,40.64,162.32,139.45,115.82,69.20,180.29,324.12,222.38
1253,2020-02-04,124.10,41.32,164.77,143.94,118.55,68.51,183.58,329.06,227.47
1254,2020-02-05,126.34,41.78,167.31,146.80,121.56,68.79,184.59,332.86,228.22
1255,2020-02-06,126.45,41.99,166.93,146.90,122.47,68.81,185.73,333.98,230.19


In [7]:
#New Index Col for ETF Data frame 
equity_clean_2 = equity_clean.set_index('Date')
equity_clean_2.head()

,VOOV,EZU,IWM,SMH,IBB,XLU,VOOG,SPY,QQQ
Date,,,,,,,,,
2015-02-12,90.75,38.20,120.81,56.06,106.81,45.87,102.60,208.92,106.00
2015-02-13,91.08,38.29,121.53,56.39,107.82,45.17,103.12,209.78,106.91
2015-02-17,91.13,38.34,121.91,56.63,108.94,45.13,103.24,210.11,107.01
2015-02-18,91.01,38.44,122.04,56.44,109.81,46.20,103.44,210.13,107.16
2015-02-19,90.75,38.40,122.05,56.47,110.66,45.68,103.64,209.98,107.69


In [8]:
#Resetting the Dataframe with the dropped column
new_first_equity = equity_clean_2.reset_index()
new_first_equity

,Date,VOOV,EZU,IWM,SMH,IBB,XLU,VOOG,SPY,QQQ
0,2015-02-12,90.75,38.20,120.81,56.06,106.81,45.87,102.60,208.92,106.00
1,2015-02-13,91.08,38.29,121.53,56.39,107.82,45.17,103.12,209.78,106.91
2,2015-02-17,91.13,38.34,121.91,56.63,108.94,45.13,103.24,210.11,107.01
3,2015-02-18,91.01,38.44,122.04,56.44,109.81,46.20,103.44,210.13,107.16
4,2015-02-19,90.75,38.40,122.05,56.47,110.66,45.68,103.64,209.98,107.69
...,...,...,...,...,...,...,...,...,...,...
1251,2020-02-03,122.67,40.64,162.32,139.45,115.82,69.20,180.29,324.12,222.38
1252,2020-02-04,124.10,41.32,164.77,143.94,118.55,68.51,183.58,329.06,227.47
1253,2020-02-05,126.34,41.78,167.31,146.80,121.56,68.79,184.59,332.86,228.22
1254,2020-02-06,126.45,41.99,166.93,146.90,122.47,68.81,185.73,333.98,230.19


In [27]:
#Read In Second Equity Dataframe Prices
second_equity_rtns = pd.read_csv('final_equity_returns.csv')
second_equity_rtns.head()
second_equity_rtns2 = second_equity_rtns.drop(columns=['Date'])
second_equity_rtns2.head()

,BKF,EEM,AIA
0,36.46,40.21,48.81
1,36.77,40.67,49.06
2,36.87,40.58,49.16
3,36.98,40.57,49.18
4,36.86,40.40,49.03


In [10]:
#Read In Fixed Income Dataframe Prices
fixed_income_rtns = pd.read_csv('final_fixed_income.csv')
fixed_income_rtns.head()

,Date,AGG,TLT,SHY,IEI,HYG,BKLN,LQD
0,2015-02-11,110.87,129.95,84.61,123.09,91.00,24.07,121.00
1,2015-02-12,110.89,129.52,84.64,123.23,90.99,24.10,121.31
2,2015-02-13,110.76,128.19,84.64,123.09,91.12,24.02,120.83
3,2015-02-17,110.31,126.26,84.60,122.64,91.06,24.12,120.27
4,2015-02-18,110.62,127.01,84.70,123.08,91.37,24.11,120.70


In [11]:
#Dropping the First Row in the Fixed Income DataFrame
fixed_income_final = fixed_income_rtns.drop([0])
fixed_income_final

,Date,AGG,TLT,SHY,IEI,HYG,BKLN,LQD
1,2015-02-12,110.89,129.52,84.64,123.23,90.99,24.10,121.31
2,2015-02-13,110.76,128.19,84.64,123.09,91.12,24.02,120.83
3,2015-02-17,110.31,126.26,84.60,122.64,91.06,24.12,120.27
4,2015-02-18,110.62,127.01,84.70,123.08,91.37,24.11,120.70
5,2015-02-19,110.53,126.17,84.65,122.86,91.16,24.14,120.68
...,...,...,...,...,...,...,...,...
1252,2020-02-03,114.37,145.54,84.96,127.73,87.15,22.61,130.65
1253,2020-02-04,113.99,143.58,84.87,127.35,87.67,22.62,130.33
1254,2020-02-05,113.82,142.01,84.81,127.08,88.03,22.62,130.25
1255,2020-02-06,113.89,142.65,84.83,127.09,88.10,22.60,130.19


In [12]:
#New Index Col for SPY Data frame 
fixed_income_clean = fixed_income_final.set_index('Date')
fixed_income_clean.head()

,AGG,TLT,SHY,IEI,HYG,BKLN,LQD
Date,,,,,,,
2015-02-12,110.89,129.52,84.64,123.23,90.99,24.10,121.31
2015-02-13,110.76,128.19,84.64,123.09,91.12,24.02,120.83
2015-02-17,110.31,126.26,84.60,122.64,91.06,24.12,120.27
2015-02-18,110.62,127.01,84.70,123.08,91.37,24.11,120.70
2015-02-19,110.53,126.17,84.65,122.86,91.16,24.14,120.68


In [13]:
#Resetting the Dataframe with the dropped column
new_fixed_income = fixed_income_clean.reset_index()
new_fixed_income

,Date,AGG,TLT,SHY,IEI,HYG,BKLN,LQD
0,2015-02-12,110.89,129.52,84.64,123.23,90.99,24.10,121.31
1,2015-02-13,110.76,128.19,84.64,123.09,91.12,24.02,120.83
2,2015-02-17,110.31,126.26,84.60,122.64,91.06,24.12,120.27
3,2015-02-18,110.62,127.01,84.70,123.08,91.37,24.11,120.70
4,2015-02-19,110.53,126.17,84.65,122.86,91.16,24.14,120.68
...,...,...,...,...,...,...,...,...
1251,2020-02-03,114.37,145.54,84.96,127.73,87.15,22.61,130.65
1252,2020-02-04,113.99,143.58,84.87,127.35,87.67,22.62,130.33
1253,2020-02-05,113.82,142.01,84.81,127.08,88.03,22.62,130.25
1254,2020-02-06,113.89,142.65,84.83,127.09,88.10,22.60,130.19


In [14]:
#Dropping the Date Column in Fixed Income DataFrame 
new_fixed_income_1 = new_fixed_income.drop(columns = ['Date'])
new_fixed_income_1

,AGG,TLT,SHY,IEI,HYG,BKLN,LQD
0,110.89,129.52,84.64,123.23,90.99,24.10,121.31
1,110.76,128.19,84.64,123.09,91.12,24.02,120.83
2,110.31,126.26,84.60,122.64,91.06,24.12,120.27
3,110.62,127.01,84.70,123.08,91.37,24.11,120.70
4,110.53,126.17,84.65,122.86,91.16,24.14,120.68
...,...,...,...,...,...,...,...
1251,114.37,145.54,84.96,127.73,87.15,22.61,130.65
1252,113.99,143.58,84.87,127.35,87.67,22.62,130.33
1253,113.82,142.01,84.81,127.08,88.03,22.62,130.25
1254,113.89,142.65,84.83,127.09,88.10,22.60,130.19


In [20]:
#Concat all DataFrames for Full Universe
rtns_universe = pd.concat([new_first_equity,second_equity_rtns,new_fixed_income_1],axis = 'columns', join = 'inner')
rtns_universe.set_index('Date', inplace=True)
rtns_universe.head()

,VOOV,EZU,IWM,SMH,IBB,XLU,VOOG,SPY,QQQ,BKF,EEM,AIA,AGG,TLT,SHY,IEI,HYG,BKLN,LQD
Date,,,,,,,,,,,,,,,,,,,
"(2015-02-12, 2015-02-12)",90.75,38.20,120.81,56.06,106.81,45.87,102.60,208.92,106.00,36.46,40.21,48.81,110.89,129.52,84.64,123.23,90.99,24.10,121.31
"(2015-02-13, 2015-02-13)",91.08,38.29,121.53,56.39,107.82,45.17,103.12,209.78,106.91,36.77,40.67,49.06,110.76,128.19,84.64,123.09,91.12,24.02,120.83
"(2015-02-17, 2015-02-17)",91.13,38.34,121.91,56.63,108.94,45.13,103.24,210.11,107.01,36.87,40.58,49.16,110.31,126.26,84.60,122.64,91.06,24.12,120.27
"(2015-02-18, 2015-02-18)",91.01,38.44,122.04,56.44,109.81,46.20,103.44,210.13,107.16,36.98,40.57,49.18,110.62,127.01,84.70,123.08,91.37,24.11,120.70
"(2015-02-19, 2015-02-19)",90.75,38.40,122.05,56.47,110.66,45.68,103.64,209.98,107.69,36.86,40.40,49.03,110.53,126.17,84.65,122.86,91.16,24.14,120.68


In [28]:
equity_universe = pd.concat([new_first_equity,second_equity_rtns2],axis = 'columns', join = 'inner')
equity_universe.set_index('Date', inplace=True)
equity_universe.head()

,VOOV,EZU,IWM,SMH,IBB,XLU,VOOG,SPY,QQQ,BKF,EEM,AIA
Date,,,,,,,,,,,,
2015-02-12,90.75,38.20,120.81,56.06,106.81,45.87,102.60,208.92,106.00,36.46,40.21,48.81
2015-02-13,91.08,38.29,121.53,56.39,107.82,45.17,103.12,209.78,106.91,36.77,40.67,49.06
2015-02-17,91.13,38.34,121.91,56.63,108.94,45.13,103.24,210.11,107.01,36.87,40.58,49.16
2015-02-18,91.01,38.44,122.04,56.44,109.81,46.20,103.44,210.13,107.16,36.98,40.57,49.18
2015-02-19,90.75,38.40,122.05,56.47,110.66,45.68,103.64,209.98,107.69,36.86,40.40,49.03


In [32]:
equity_universe.corr()
#equity_universe_corr = equity_universe.corr()
#sns.heatmap(equity_universe_corr, vmin=-1, vmax=1)


,VOOV,EZU,IWM,SMH,IBB,XLU,VOOG,SPY,QQQ,BKF,EEM,AIA
VOOV,1.000000,0.670407,0.943539,0.978427,0.360724,0.853446,0.958094,0.982167,0.960257,0.837631,0.767242,0.873112
EZU,0.670407,1.000000,0.741203,0.648990,0.591585,0.349109,0.592971,0.629062,0.599213,0.889252,0.922454,0.884312
IWM,0.943539,0.741203,1.000000,0.924481,0.453900,0.700814,0.915238,0.935090,0.917559,0.870080,0.819911,0.913210
SMH,0.978427,0.648990,0.924481,1.000000,0.318366,0.870135,0.974058,0.985197,0.980412,0.824959,0.744749,0.871227
IBB,0.360724,0.591585,0.453900,0.318366,1.000000,0.061177,0.368475,0.370043,0.366999,0.531972,0.500252,0.452748
XLU,0.853446,0.349109,0.700814,0.870135,0.061177,1.000000,0.869204,0.870709,0.866128,0.571795,0.475822,0.615771
VOOG,0.958094,0.592971,0.915238,0.974058,0.368475,0.869204,1.000000,0.994772,0.998198,0.780804,0.682368,0.821305
SPY,0.982167,0.629062,0.935090,0.985197,0.370043,0.870709,0.994772,1.000000,0.994483,0.810119,0.721238,0.849682
QQQ,0.960257,0.599213,0.917559,0.980412,0.366999,0.866128,0.998198,0.994483,1.000000,0.781591,0.684257,0.825686
BKF,0.837631,0.889252,0.870080,0.824959,0.531972,0.571795,0.780804,0.810119,0.781591,1.000000,0.980898,0.979374


In [35]:
# Calculate expected returns and sample covariance
mu = expected_returns.mean_historical_return(equity_universe)
S = risk_models.sample_cov(equity_universe)

In [40]:
# Optimise for maximal Sharpe ratio This is our effictient frontier for equities
ef = EfficientFrontier(mu, S)
raw_weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()
ef.save_weights_to_file("equity_efficient.csv")  # saves to file
print(cleaned_weights)
ef.portfolio_performance(verbose=True)

{'VOOV': 0.0, 'EZU': 0.0, 'IWM': 0.0, 'SMH': 0.28993, 'IBB': 0.0, 'XLU': 0.36465, 'VOOG': 0.0, 'SPY': 0.0, 'QQQ': 0.34542, 'BKF': 0.0, 'EEM': 0.0, 'AIA': 0.0}
Expected annual return: 15.4%
Annual volatility: 14.0%
Sharpe Ratio: 0.96


(0.15422025816022122, 0.1397015717502023, 0.9607641236865816)

In [51]:
# Equally WEighted Equity Portfolio
mu = expected_returns.mean_historical_return(equity_universe)
S = risk_models.sample_cov(equity_universe)

ef = EfficientFrontier(mu, S, weight_bounds = (0.01, 0.0833))
raw_weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()
ef.save_weights_to_file("equity_balanced.csv")  # saves to file
print(cleaned_weights)
ef.portfolio_performance(verbose=True)

{'VOOV': 0.0833, 'EZU': 0.0833, 'IWM': 0.0833, 'SMH': 0.0833, 'IBB': 0.0833, 'XLU': 0.0833, 'VOOG': 0.0833, 'SPY': 0.0833, 'QQQ': 0.0833, 'BKF': 0.0833, 'EEM': 0.0833, 'AIA': 0.0833}
Expected annual return: 9.2%
Annual volatility: 14.4%
Sharpe Ratio: 0.50


(0.09243634599641837, 0.143658477699061, 0.5042260446902385)

In [50]:
# Fixed income Optimized Portfolio Yoda
#new_fixed_income.set_index('Date', inplace=True)
mu = expected_returns.mean_historical_return(new_fixed_income)
S = risk_models.sample_cov(new_fixed_income)

ef = EfficientFrontier(mu, S)
raw_weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()
ef.save_weights_to_file("fixed_income_optimized.csv")  # saves to file
print(cleaned_weights)
ef.portfolio_performance(verbose=True)

{'AGG': 0.0, 'TLT': 1.0, 'SHY': 0.0, 'IEI': 0.0, 'HYG': 0.0, 'BKLN': 0.0, 'LQD': 0.0}
Expected annual return: 2.9%
Annual volatility: 12.0%
Sharpe Ratio: 0.08


(0.029111893104804918, 0.1202488554967841, 0.07577530004057796)

In [44]:
#new_fixed_income.corr()

,AGG,TLT,SHY,IEI,HYG,BKLN,LQD
AGG,1.000000,0.916708,0.853412,0.956879,0.232767,-0.009468,0.877537
TLT,0.916708,1.000000,0.637786,0.871337,0.055094,-0.257475,0.870690
SHY,0.853412,0.637786,1.000000,0.912271,0.088816,0.116667,0.540194
IEI,0.956879,0.871337,0.912271,1.000000,0.019986,-0.152535,0.750777
HYG,0.232767,0.055094,0.088816,0.019986,1.000000,0.745337,0.437243
BKLN,-0.009468,-0.257475,0.116667,-0.152535,0.745337,1.000000,-0.026671
LQD,0.877537,0.870690,0.540194,0.750777,0.437243,-0.026671,1.000000


In [48]:
# Fixed income Equally Weighted
#new_fixed_income.set_index('Date', inplace=True)
mu = expected_returns.mean_historical_return(new_fixed_income)
S = risk_models.sample_cov(new_fixed_income)

ef = EfficientFrontier(mu, S, weight_bounds = (0.00, 0.143)) 
raw_weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()
ef.save_weights_to_file("fixed_income_equal_weighted.csv")  # saves to file
print(cleaned_weights)
ef.portfolio_performance(verbose=True)

{'AGG': 0.143, 'TLT': 0.143, 'SHY': 0.143, 'IEI': 0.143, 'HYG': 0.143, 'BKLN': 0.142, 'LQD': 0.143}
Expected annual return: 0.6%
Annual volatility: 3.4%
Sharpe Ratio: -0.41


(0.0060303938096676595, 0.03366581590535977, -0.41494928355823124)

In [52]:
# Blended - Will the real Yoda please stand up!

mu = expected_returns.mean_historical_return(rtns_universe)
S = risk_models.sample_cov(rtns_universe)

ef = EfficientFrontier(mu, S) 
raw_weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()
ef.save_weights_to_file("blended_optimized.csv")  # saves to file
print(cleaned_weights)
ef.portfolio_performance(verbose=True)

{'VOOV': 0.0, 'EZU': 0.0, 'IWM': 0.0, 'SMH': 0.22876, 'IBB': 0.0, 'XLU': 0.20055, 'VOOG': 0.0, 'SPY': 0.0, 'QQQ': 0.31059, 'BKF': 0.0, 'EEM': 0.0, 'AIA': 0.0, 'AGG': 0.0, 'TLT': 0.2601, 'SHY': 0.0, 'IEI': 0.0, 'HYG': 0.0, 'BKLN': 0.0, 'LQD': 0.0}
Expected annual return: 12.8%
Annual volatility: 10.8%
Sharpe Ratio: 1.00


(0.1278213224626065, 0.10832927809160244, 0.9953110032860515)

In [63]:
# Maximized Profit Adjusting Returns 
mu = expected_returns.mean_historical_return(rtns_universe)
S = risk_models.sample_cov(rtns_universe)

ef = EfficientFrontier(mu, S, weight_bounds =(-1, 1)) 
weights = ef.efficient_return(target_return = 0.20, market_neutral = True)
#cleaned_weights = ef.clean_weights()
#ef.save_weights_to_file("blended_optimized.csv")  # saves to file
print(cleaned_weights)
ef.portfolio_performance(verbose=True)

{'VOOV': 0.0, 'EZU': 0.0, 'IWM': 0.0, 'SMH': 0.22876, 'IBB': 0.0, 'XLU': 0.20055, 'VOOG': 0.0, 'SPY': 0.0, 'QQQ': 0.31059, 'BKF': 0.0, 'EEM': 0.0, 'AIA': 0.0, 'AGG': 0.0, 'TLT': 0.2601, 'SHY': 0.0, 'IEI': 0.0, 'HYG': 0.0, 'BKLN': 0.0, 'LQD': 0.0}
Expected annual return: 20.0%
Annual volatility: 10.0%
Sharpe Ratio: 1.80


(0.20000000000949542, 0.0999968895444766, 1.8000559900359208)